### The goal of this notebook is to measure the Right or Left stimulus from the firing rate of clusters across the time period before and after stimlus onset. I did the analysis for one probe example with around 94 channels in V1. 

#  Z-score firing rate in right / left trials for each cluster and time 

In [ ]:

import pandas as pd
import numpy as np
from get_data import get_behavior, get_channels, get_spikes
from firing_rate import firing_Rates_onClusters

# an example pid and eid 
pid = 'a3d13b05-bf4d-427a-a2d5-2fe050d603ec' # probe id (94 channels in V1)
eid = '03cf52f6-fba6-4743-a42e-dd1ac3072343' # experiment id

######################
# Load data
########################
# Load behavior
behavior = get_behavior(eid, mode='download')  # download the data from the IBL database or 'load' data from the local directory
right_onset = behavior[behavior['contrastRight'] == 1]['stimOn_times']
left_onset = behavior[behavior['contrastLeft'] == 1]['stimOn_times']
stim_events = {'right': right_onset, 'left': left_onset}

# load channel data 
channels = get_channels(pid, eid,  mode='download')


# load spikes
spikes_datasets = get_spikes(pid, mode='download', path=None)
spikes = spikes_datasets['spikes']
spike_times = spikes['times']
spike_clusters = spikes['clusters']
kp_idx = np.where(~np.isnan(spike_clusters))[0] # Remove any nan depths
spike_times = spike_times[kp_idx]
spike_clusters = spike_clusters[kp_idx]


# select only the clusters in V1 
clusters_dataset = spikes_datasets['clusters']
cluster_channels = clusters_dataset['channels']
cluster_names = channels['acronym'][cluster_channels]
cluster_names = cluster_names.reset_index(drop=True)# reset the index 

valid_regions = ['VISp1', 'VISp2/3', 'VISp4', 'VISp5', 'VISp6a', 'VISp6b']
valid_cluster_idx = np.where(np.isin(cluster_names, valid_regions))[0]
valid_channels = cluster_channels[valid_cluster_idx]
if len(valid_cluster_idx) == 0:
    print('No clusters found in V1 regions.')
    exit()
# Filter spike data for valid clusters
valid_spike_mask = np.isin(spike_clusters, valid_cluster_idx)
spike_times = spike_times[valid_spike_mask]
spike_clusters = spike_clusters[valid_spike_mask]


######################
# compute firing rate
########################
z_score_firing_rate, times, depths = firing_Rates_onClusters(stim_events, spike_times, spike_clusters,  t_bin=0.1, pre_stim=0.4, post_stim=1)
z_score_right = z_score_firing_rate['right']
z_score_left = z_score_firing_rate['left']

# average firing rate for right and left trials
right_activity_mean = np.mean(z_score_right, axis=0)
left_activity_mean = np.mean(z_score_left, axis=0)




# plot the average firing rate in right / left trials for each depth and time

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
from brainbox.ephys_plots import plot_brain_regions





# Create a figure with GridSpec for multiple plots
fig = plt.figure(figsize=(12, 10))
gs = GridSpec(2, 2, width_ratios=[4, 1])  # 2 rows, 2 columns

# First column, first row: Right Trials
ax1 = fig.add_subplot(gs[0, 0])
im1 = ax1.imshow(right_activity_mean, aspect='auto', origin='lower',
                  extent=[times[0], times[-1], depths[0], depths[-1]], 
                  cmap='viridis')
ax1.set_title('Firing Rate for Right Trials')
ax1.set_ylabel('clusters id')

plt.colorbar(im1, ax=ax1, label='Z-score Firing Rate')

# Second column, first row: Brain Regions for Right Trials
ax2 = fig.add_subplot(gs[0, 1])  # Second column, first row
plot_brain_regions(valid_channels, display=True, ax=ax2, 
                   title='Brain Regions for Right Trials', label='right')

# First column, second row: Left Trials
ax3 = fig.add_subplot(gs[1, 0])
im2 = ax3.imshow(left_activity_mean, aspect='auto', origin='lower',
                  extent=[times[0], times[-1], depths[0], depths[-1]], 
                  cmap='viridis')
ax3.set_title('Firing Rate for Left Trials')
ax3.set_xlabel('Time (s)')
ax3.set_ylabel('clusters id')

plt.colorbar(im2, ax=ax3, label='Z-score Firing Rate')

# Second column, second row: Brain Regions for Left Trials
ax4 = fig.add_subplot(gs[1, 1])  # Second column, second row
plot_brain_regions(valid_channels,  display=True, ax=ax4, 
                   title='Brain Regions for Left Trials', label='left')

# Layout adjustment
plt.tight_layout()  # Adjust layout to prevent clipping
plt.show()


# compute the decoding accuracy for each cluster and time bins using AU-ROC

 #### AU-ROC values higher than 0.5   indicate better decoding right stim 
 #### AU-ROC values lower than 0.5   indicate better decoding left stim

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample

#  z_score_right and z_score_left are already available with shape (trials, depths, time)
# Get the number of trials for right and left
n_right_trials, n_clusters, n_time = z_score_right.shape
n_left_trials = z_score_left.shape[0]

# Initialize a storage for AUROC values
auroc_values = np.zeros((n_clusters, n_time))

for cluster in range(n_clusters):
    for time in range(n_time):
        # Prepare the feature set and labels
        firing_rates = np.concatenate((z_score_right[:, cluster, time], z_score_left[:, cluster, time]))
        labels = np.concatenate((np.ones(n_right_trials), np.zeros(n_left_trials)))  # 1 for right, 0 for left

        # Handle class imbalance: downsample the majority class (right trials)
        if n_right_trials > n_left_trials:
            firing_rates, labels = resample(firing_rates, labels, 
                                             replace=False, 
                                             n_samples=n_left_trials, 
                                             random_state=42)
        
        # Fit logistic regression
        model = LogisticRegression()
        model.fit(firing_rates.reshape(-1, 1), labels)  # Reshape for a single feature

        # Make predictions and calculate AUROC
        probabilities = model.predict_proba(firing_rates.reshape(-1, 1))[:, 1]
        auroc = roc_auc_score(labels, probabilities)
        
        # Store the AUROC value
        auroc_values[cluster, time] = auroc


# plot the decoding performance for each depth and time

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
from brainbox.ephys_plots import plot_brain_regions


# Create a figure with GridSpec for multiple plots
fig = plt.figure(figsize=(18, 8))
gs = GridSpec(1, 2, width_ratios=[4, 1])  # 1 row, 2 columns

# First column: AUROC Heatmap
ax1 = fig.add_subplot(gs[0, 0])
sns.heatmap(auroc_values, cmap='coolwarm', center=0.5, cbar_kws={'label': 'AUROC'}, ax=ax1)
ax1.set_title('AUROC Values for Right vs. Left Trials')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('clusters id')

# Set x ticks
ax1.set_xticks(np.arange(len(times)))
ax1.set_xticklabels(np.round(times, 2), rotation=45)

# # Set y ticks every 10 units
# y_ticks_indices = np.arange(0, len(depths), 10)
# ax1.set_yticks(y_ticks_indices)
# ax1.set_yticklabels(depths[y_ticks_indices])
ax1.invert_yaxis()
# Second column: Plot Brain Regions
ax2 = fig.add_subplot(gs[0, 1])  # Second column
# Assuming you have the necessary parameters: ids and true_depths
plot_brain_regions( valid_channels, display=True, ax=ax2, 
                   title='Brain Regions', label='right')

# Layout adjustment
plt.tight_layout()  # Adjust layout to prevent clipping
plt.show()
